In [1]:
import requests
import bs4
import pandas as pd
import datetime
import os

In [2]:
def getWebdata():
    page =requests.get('https://fred.stlouisfed.org/categories/')
    beatiful = bs4.BeautifulSoup(page.content, 'html.parser')
    category_page =beatiful.find_all("div",{"class":"fred-categories-group"})        
    return category_page

In [10]:
def getsubpage(id):        
    sub_url =f'https://fred.stlouisfed.org/categories/{id}'
    subpage =requests.get(sub_url)
    beatiful = bs4.BeautifulSoup(subpage.content, 'html.parser')
    category_page_col =beatiful.find_all("div",{"class":"col-12 panel-menu"})
    category_page_subcol =beatiful.find_all("div",{"class":"col-12 subcats"})  
    return category_page_col,category_page_subcol

In [3]:
category_page = getWebdata()

In [4]:
category_page

[<div class="fred-categories-group">
 <p class="large fred-categories-parent">
 <a href="/categories/32991"><strong>Money, Banking, &amp; Finance</strong></a> <span class="category-count">(13,000+)</span>
 </p>
 <p class="fred-categories-children">
 <span class="fred-categories-child"><a href="/categories/22">Interest Rates</a> <span class="category-count">(960+)</span></span>
 <span class="fred-categories-child"><a href="/categories/15">Exchange Rates</a> <span class="category-count">(9+)</span></span>
 <span class="fred-categories-child"><a href="/categories/24">Monetary Data</a> <span class="category-count">(1,100+)</span></span>
 <span class="fred-categories-child"><a href="/categories/46">Financial Indicators</a> <span class="category-count">(2,300+)</span></span>
 <span class="fred-categories-child"><a href="/categories/23">Banking</a> <span class="category-count">(5,200+)</span></span>
 <span class="fred-categories-child"><a href="/categories/32360">Business Lending</a> <span cl

In [9]:
for i in category_page:
    line_ref = i.find("a")
    print(line_ref)
    print("===========")
    pa_id = line_ref['href'][line_ref['href'].rfind('/')+1:]
    print(pa_id)
    pa_name = line_ref.text
    print(pa_name)
    break

<a href="/categories/32991"><strong>Money, Banking, &amp; Finance</strong></a>
32991
Money, Banking, & Finance


In [11]:
category_page = getWebdata()
category = []
for i in category_page:    
    line_ref = i.find("a")
    pa_id = line_ref['href'][line_ref['href'].rfind('/')+1:]
    pa_name = line_ref.text
    category.append((pa_name,pa_id,None,None))
    for a in i:
        for o in a:
            _temp = o.find('a')
            if _temp != -1 and _temp !=None:
                name= _temp.text
                id  =_temp['href'][_temp['href'].rfind('/')+1:]
                category.append((name,id,pa_name,pa_id))
                category_page_col,category_page_subcol = getsubpage(id)                                      
                if category_page_subcol:
                    for i in category_page_subcol:
                        sub_page = i.find_all('a',href=True)
                        for item in sub_page:
                            subname = item.text
                            subid   = item['href'][item['href'].rfind('/')+1:]
                            category.append((subname,subid,name,id)) 
                elif category_page_col:
                    for i in category_page_col:
                        sub_page = i.find_all('a',href=True)
                        for item in sub_page:
                            subname = item.text
                            subid   = item['href'][item['href'].rfind('/')+1:]
                            category.append((subname,subid,name,id))

In [12]:
category

[('Money, Banking, & Finance', '32991', None, None),
 ('Interest Rates', '22', 'Money, Banking, & Finance', '32991'),
 ('AMERIBOR Benchmark Rates', '34009', 'Interest Rates', '22'),
 ('Automobile Loan Rates', '33058', 'Interest Rates', '22'),
 ('Bankers Acceptance Rate', '51', 'Interest Rates', '22'),
 ('Certificates of Deposit', '121', 'Interest Rates', '22'),
 ('Commercial Paper', '120', 'Interest Rates', '22'),
 ('Corporate Bonds', '32348', 'Interest Rates', '22'),
 ('Credit Card Loan Rates', '33059', 'Interest Rates', '22'),
 ('EONIA Rates', '34005', 'Interest Rates', '22'),
 ('Euro Short-Term Rate', '34007', 'Interest Rates', '22'),
 ('Eurodollar Deposits', '32298', 'Interest Rates', '22'),
 ('FRB Rates - discount, fed funds, primary credit',
  '118',
  'Interest Rates',
  '22'),
 ('Interest Checking Accounts', '33056', 'Interest Rates', '22'),
 ('Interest Rate Spreads', '33446', 'Interest Rates', '22'),
 ('Interest Rate Swaps', '32299', 'Interest Rates', '22'),
 ('Long-Term Secur

In [13]:
df_category = pd.DataFrame(category,columns=['name','id','parent_name','parent_id'])
df_category[['name','parent_name']] = df_category[['name','parent_name']].astype('string')
df_category[['id','parent_id']] = df_category[['id','parent_id']].fillna(0).astype('int')

In [14]:
df_category.head()

,name,id,parent_name,parent_id
0,"Money, Banking, & Finance",32991,<NA>,0
1,Interest Rates,22,"Money, Banking, & Finance",32991
2,AMERIBOR Benchmark Rates,34009,Interest Rates,22
3,Automobile Loan Rates,33058,Interest Rates,22
4,Bankers Acceptance Rate,51,Interest Rates,22


In [24]:
import requests
import json

def get_json(category_id):
    url = f"https://api.stlouisfed.org/fred/category/children?category_id={category_id}&api_key=abcdefghijklmnopqrstuvwxyz123456&file_type=json"
    response = requests.get(url)
    # Kiểm tra nếu yêu cầu thành công
    if response.status_code == 200:
        data = json.loads(response.text)
        return data
    else:
        print("Không thể lấy dữ liệu, mã lỗi:", response.status_code)


In [28]:
category_id=23
url = f"https://api.stlouisfed.org/fred/category/children?category_id={category_id}&api_key=b190fd48c808c7d3be5c40454f3e57f3&file_type=json"

response = requests.get(url)
data = json.loads(response.text)
data

{'categories': [{'id': 34078, 'name': 'Banking Indexes', 'parent_id': 23},
  {'id': 100, 'name': 'Commercial Banking', 'parent_id': 23},
  {'id': 83,
   'name': 'Condition of Banks',
   'parent_id': 23,
   'notes': 'For census division definitions, see http://research.stlouisfed.org/fred2/censusdivisions.'},
  {'id': 101, 'name': 'Consumer Credit', 'parent_id': 23},
  {'id': 32440,
   'name': 'Delinquencies and Delinquency Rates',
   'parent_id': 23},
  {'id': 33121,
   'name': 'Failures and Assistance Transactions',
   'parent_id': 23},
  {'id': 64, 'name': '8th District Banking Performance', 'parent_id': 23},
  {'id': 33445, 'name': 'Mortgage Debt Outstanding', 'parent_id': 23},
  {'id': 32439,
   'name': 'Net Charge-Offs and Charge-Off Rates',
   'parent_id': 23},
  {'id': 99, 'name': 'Securities & Investments', 'parent_id': 23},
  {'id': 34111,
   'name': 'Senior Credit Officer Opinion Survey',
   'parent_id': 23},
  {'id': 32239, 'name': 'Senior Loan Officer Survey', 'parent_id': 